In [0]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import operator

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from collections import defaultdict
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import sys
from google.colab import drive
drive.mount('/content/gdrive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
TRAIN_DATA_FILE= 'gdrive/My Drive/dl/data/train.csv'

train_df = pd.read_csv(TRAIN_DATA_FILE)
print(train_df.columns)
train_df = train_df.drop(['id', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1)
print(train_df.columns)

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')
Index(['comment_text', 'toxic'], dtype='object')


In [0]:
def clean_text(text):
    special_character_removal=re.compile(r'[^?!a-z ]',re.IGNORECASE)
    replace_numbers=re.compile(r'\d+',re.IGNORECASE)

    text = text.lower()
    text = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", "", text)
    text = re.sub(r"(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}", "", text)
      
    text = re.sub(r"image:[a-zA-Z0-9]*\.jpg", " ", text)
    text = re.sub(r"image:[a-zA-Z0-9]*\.png", " ", text)
    text = re.sub(r"image:[a-zA-Z0-9]*\.bmp", " ", text)
    text = re.sub(r"image:[a-zA-Z0-9]*\.svg", " ", text)
    text = re.sub(r"image:[a-zA-Z0-9]*\.gif", " ", text)

    text = re.sub(r"\n", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"f*ck", "fuck", text)
    text = replace_numbers.sub(' ', text)
    text = special_character_removal.sub('',text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    
    if text == '':
      text = 'nice'

    return (text)

In [0]:
list_sentences_train = train_df["comment_text"].fillna("no comment").values

In [0]:
import tqdm
comments = [clean_text(text) for text in tqdm.tqdm(list_sentences_train)]

100%|██████████| 159571/159571 [02:45<00:00, 962.09it/s]


In [0]:
train_df['comment_text'] = comments
train_df.to_csv('gdrive/My Drive/dl/data/clean_train.csv', index=False)

In [0]:
curses = set()

for text, label in zip(train_df['comment_text'],train_df['toxic']):
  if(label == 1):
    for word in text.split():
      curses.add(word)
print(len(curses))
print(curses)
      

for text, label in zip(train_df['comment_text'],train_df['toxic']):
  if(label == 0):
    for word in text.split((' ')):
      curses.discard(word)
print(len(curses))
print(curses)

25296
{'hollywood', 'gatekeep', 'warkosign', 'thjoker', 'negrito', 'connor', 'wikipidea', 'nissim', 'regiment', 'schingslong', 'demonstr', 'prissi', 'come', 'eupham', 'devolv', 'disadvantag', 'mayt', 'etiti', 'espionag', 'fingerwag', 'hooooooooorrid', 'tactic', 'sheffield', 'angelmcremili', 'rifel', 'filmographi', 'aryan', 'wikpedian', 'meghan', 'from', 'enochlau', 'cia', 'yoke', 'challeng', 'deek', 'skull', 'hispan', 'contador', 'destruct', 'shite', 'pashtu', 'saaaaad', 'chozen', 'anetod', 'dank', 'precious', 'jafucknif', 'societi', 'clark', 'wikipdeia', 'vassal', 'lender', 'shrine', 'nrdg', 'apr', 'artlc', 'pathet', 'inund', 'pan', 'like', 'jibberish', 'goug', 'confetti', 'fluid', 'tension', 'intimid', 'aboutmovi', 'motorcycl', 'tof', 'dress', 'defent', 'kassam', 'hippi', 'spite', 'groundbreak', 'saruj', 'comeon', 'shithol', 'gayyyyyyyyyyy', 'leadersfavorit', 'vegetasaiyan', 'lampard', 'herein', 'saturn', 'haaaaaaa', 'propagandist', 'johnboywalton', 'foreskin', 'stall', 'mgiganteus',

In [0]:
print(sum(train_df['toxic']))
print(train_df['comment_text'][36450])

15294
classwikit damn right mark ! ! ! serv you right dont do bad hting !
